# Housing Data Workflow Notebook

Modular workflow where you can run individual steps independently.
Run cells in order or skip any steps you don't need.

Each step shows dataframe views and statistics for inspection.

## Quick Start
- Run **Setup** cell first
- Then run any combination of Step 1-4 cells
- Skip cells you don't want to execute
- Each cell is self-contained and shows results

## 🔧 Setup

Run this cell first to import modules and define helper functions.

In [1]:
import sys
from pathlib import Path
from typing import Optional
import time

import pandas as pd

# Add current directory to path for local imports
sys.path.append(".")

# Import our workflow modules
from fetch_affordable_housing_data import update_local_data, verify_and_fetch_hpd_data
from query_ll44_funding import query_and_add_financing
from query_dob_filings import query_dob_filings
from query_co_filings import query_co_filings
from HPD_DOB_Join_On_BIN import create_separate_timelines
from create_timeline_chart import create_timeline_chart, create_financing_charts
from data_quality import quality_tracker

print("✅ All imports successful")

# Helper functions
def _normalize_bin(bin_value) -> Optional[str]:
    """Normalize BIN to a clean string."""
    if pd.isna(bin_value):
        return None
    try:
        return str(int(float(bin_value)))
    except (TypeError, ValueError):
        value = str(bin_value).strip()
        return value or None

def _write_bin_file(source_csv: Path, output_txt: Path) -> Path:
    """Extract BINs from a CSV and write them to a text file for CO searches."""
    df = pd.read_csv(source_csv)
    candidate_cols = [col for col in df.columns if col.lower() in ("bin", "bin_normalized")]
    if not candidate_cols:
        raise SystemExit(f"Could not find a BIN column in {source_csv}")

    bins = [_normalize_bin(val) for val in df[candidate_cols[0]].dropna()]
    bins = sorted({b for b in bins if b})

    output_txt.parent.mkdir(parents=True, exist_ok=True)
    output_txt.write_text("\n".join(bins))
    print(f"Wrote {len(bins)} BINs to {output_txt}")
    return output_txt

print("✅ Helper functions defined")

✅ All imports successful
✅ Helper functions defined


## 📥 Step 1: Fetch HPD Data

Load or refresh the HPD affordable housing dataset.

**Options:**
- Set `refresh_data = True` to fetch fresh data
- Set `refresh_data = False` to use existing data

In [2]:
# Step 1 Configuration
refresh_data = False  # Set to True to fetch fresh HPD data
hpd_output_path = "data/raw/Affordable_Housing_Production_by_Building.csv"  # Output path for HPD data
refresh_hpd_projects = False  # Set to True to fetch fresh HPD projects data

print("=" * 70)
print("STEP 1: FETCH HPD DATA")
print("=" * 70)

# Start quality tracking

# Handle HPD projects cache refresh if requested
if refresh_hpd_projects:
    print("Force refreshing HPD projects cache...")
    from fetch_affordable_housing_data import verify_and_fetch_hpd_projects_data
    hpd_projects_df, hpd_projects_path = verify_and_fetch_hpd_projects_data(use_existing=False)
    print(f"HPD projects cache refreshed: {len(hpd_projects_df)} records\n")

quality_tracker.start_processing()

if refresh_data:
    print("Fetching fresh HPD data from NYC Open Data...")
    hpd_df, hpd_csv = update_local_data(hpd_output_path)
else:
    print("Verifying local HPD data against API...")
    hpd_df, hpd_csv = verify_and_fetch_hpd_data(output_path=hpd_output_path, use_projects_cache=not refresh_hpd_projects)

if not hpd_csv.exists():
    raise SystemExit(f"HPD dataset not found at {hpd_csv}")

# Record initial dataset size

# Filter to New Construction only
original_count = len(hpd_df)
hpd_df = hpd_df[hpd_df["Reporting Construction Type"] == "New Construction"].copy()
filtered_count = len(hpd_df)
filtered_out = original_count - filtered_count

print(f"🏗️ Filtered to New Construction only:")
print(f"  Original: {original_count:,} projects")
print(f"  Filtered: {filtered_count:,} projects ({filtered_count/original_count*100:.1f}%)")
print(f"  Removed: {filtered_out:,} non-new construction projects ({filtered_out/original_count*100:.1f}%)")

quality_tracker.analyze_hpd_data(hpd_df, "Full_HPD_Dataset")
quality_tracker.record_pipeline_stage("raw_hpd_data", len(hpd_df), "Raw HPD affordable housing dataset")

print(f"✅ Step 1 complete: {len(hpd_df):,} records loaded")
print(f"📁 Data location: {hpd_csv}")

# Display the dataframe
print("\n🔍 HPD Dataset Overview:")
print(f"Shape: {hpd_df.shape}")
print("\nColumns:")
for col in hpd_df.columns:
    print(f"  - {col}")

print("\n📊 Sample Data:")
display(hpd_df.head())
print("\n📈 Basic Statistics:")
display(hpd_df.describe(include="all"))

STEP 1: FETCH HPD DATA
Verifying local HPD data against API...
STEP 1: VERIFY AND FETCH HPD DATA
Found local HPD data file: data/raw/Affordable_Housing_Production_by_Building.csv
Local file has 8,604 records

Fetching 100 sample records from API for verification...
Fetching affordable housing data from NYC Open Data API...
Endpoint: https://data.cityofnewyork.us/resource/hg8x-zxpr.json
Fetching records 1-100...
  Retrieved 100 records (total: 100)

Completed! Retrieved 100 total records

Enriching building data with project-level information...
VERIFYING HPD PROJECTS CACHE
HPD projects cache file is stale (age: 0:00:16.924927) or use_existing=False
Fetching fresh data from API...
Created backup: data/processed/Affordable_Housing_Production_by_Project.csv.backup_20251129_204728
Fetching HPD Projects data from NYC Open Data API...
Endpoint: https://data.cityofnewyork.us/resource/hq68-rnsi.json
Fetching records 1-1000...
  Retrieved 1000 records (total: 1,000)
Fetching records 1001-2000..

,Project ID,Project Name,Project Start Date,Building ID,Number,Street,Borough,Postcode,BBL,BIN,...,6-BR+ Units,Unknown-BR Units,Counted Rental Units,Counted Homeownership Units,All Counted Units,Total Units,Program Group,Project Completion Date,Extended Affordability Status,Planned Tax Benefit
0,44218,MEC E. 125TH ST. PARCEL B WEST,2018-12-31T00:00:00.000,987329,2319,3 AVENUE,Manhattan,10035,1017907501,1054682,...,NaN,NaN,297.0,NaN,297.0,404,Multifamily Finance Program,2022-02-16T00:00:00.000,No,Article XI
1,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,927737,335,RALPH AVENUE,Brooklyn,11233,3015560003,3000000,...,NaN,NaN,NaN,13.0,13.0,13,Multifamily Finance Program,2024-09-24T00:00:00.000,No,Article XI
2,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,969695,35,ROCHESTER AVENUE,Brooklyn,11233,3017090009,3000000,...,NaN,NaN,NaN,8.0,8.0,8,Multifamily Finance Program,2024-09-24T00:00:00.000,No,Article XI
3,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,975702,18-22,SUYDAM PLACE,Brooklyn,11233,3017090028,3000000,...,NaN,NaN,NaN,15.0,15.0,15,Multifamily Finance Program,2024-09-24T00:00:00.000,No,Article XI
4,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,977564,329,RALPH AVENUE,Brooklyn,11233,3015560007,3000000,...,NaN,NaN,NaN,10.0,10.0,10,Multifamily Finance Program,2024-09-24T00:00:00.000,No,Article XI



📈 Basic Statistics:


,Project ID,Project Name,Project Start Date,Building ID,Number,Street,Borough,Postcode,BBL,BIN,...,6-BR+ Units,Unknown-BR Units,Counted Rental Units,Counted Homeownership Units,All Counted Units,Total Units,Program Group,Project Completion Date,Extended Affordability Status,Planned Tax Benefit
count,4216,4216,4216,3184,4216,4216,4216,3182,3115,3115,...,12.0,106.000000,3029.000000,1276.000000,4206.000000,4216.000000,4216,3297,4216,3113
unique,3749,2718,1850,3162,1965,1048,5,146,3007,2397,...,NaN,NaN,NaN,NaN,NaN,NaN,4,1437,2,8
top,53017,CONFIDENTIAL,2016-06-27T00:00:00.000,975696,----,----,Brooklyn,11239,2035150020,3000000,...,NaN,NaN,NaN,NaN,NaN,NaN,Multifamily Incentives Program,2019-05-28T00:00:00.000,No,421a
freq,83,1032,84,2,1032,1032,1873,165,9,331,...,NaN,NaN,NaN,NaN,NaN,NaN,2412,83,4214,2332
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.688679,37.684714,1.639498,27.636472,52.016129,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,6.993008,67.426568,4.116363,59.488316,107.460375,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.000000,3.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.000000,9.000000,1.000000,5.000000,10.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.000000,38.000000,1.000000,20.000000,50.000000,NaN,NaN,NaN,NaN


In [16]:
# // what are the 4 program groups

print("Program Group counts:")
print(hpd_df['Program Group'].value_counts())

print("\nTax Abatement by Program Group:")
if 'Planned Tax Benefit' in hpd_df.columns:
    tax_abate_ct = (
        hpd_df.groupby('Program Group')['Planned Tax Benefit']
        .value_counts(dropna=False)
        .unstack(fill_value=0)
        .sort_index(axis=1)
    )
    display(tax_abate_ct)
else:
    print("Column 'Planned Tax Benefit' not found in dataset.")





Program Group counts:
Program Group
Multifamily Incentives Program    2412
Homeowner Assistance Program      1027
Multifamily Finance Program        581
Small Homes Program                196
Name: count, dtype: int64

Tax Abatement by Program Group:


Planned Tax Benefit,420a,420c,421a,467-M,485-X,Article IV,Article XI,UDAAP,NaN
Program Group,,,,,,,,,
Homeowner Assistance Program,0,0,0,0,0,0,0,0,1027
Multifamily Finance Program,3,372,25,0,0,0,130,26,25
Multifamily Incentives Program,1,31,2304,2,32,0,15,0,27
Small Homes Program,0,20,3,0,0,4,19,126,24


## 💰 Step 2: Add Financing Classification

Classify projects by financing type (HPD vs Private).

**Depends on:** Step 1
**Options:**
- Set `skip_financing = True` to skip this step
- Customize output path if needed

In [10]:
# Step 2 Configuration
skip_financing = False  # Set to True to skip financing classification
refresh_ll44 = False  # Set to True to fetch fresh LL44 data
refresh_ll44_eligibility = False  # Set to True to fetch fresh LL44 eligibility data
financing_output_path = None  # Custom output path (None = auto-generate)

print("\n" + "=" * 70)
print("STEP 2: ADD FINANCING CLASSIFICATION")

# Handle LL44 cache refresh if requested
if refresh_ll44_eligibility:
    print("Force refreshing LL44 eligibility cache...")
    from query_ll44_funding import verify_and_fetch_ll44_eligibility_data
    ll44_eligibility_df, ll44_eligibility_path = verify_and_fetch_ll44_eligibility_data(use_existing=False)
    print(f"LL44 eligibility cache refreshed: {len(ll44_eligibility_df)} records\n")

if refresh_ll44:
    print("Force refreshing LL44 funding cache...")
    from query_ll44_funding import verify_and_fetch_ll44_data
    ll44_cache_df, ll44_cache_path = verify_and_fetch_ll44_data(use_existing=False)
    print(f"LL44 funding cache refreshed: {len(ll44_cache_df)} records\n")

    print("Force refreshing LL44 funding cache...")
print("=" * 70)

if skip_financing:
    print("⏭️ Skipping financing classification as requested.")
    financing_df = hpd_df.copy()
    quality_tracker.record_pipeline_stage("after_financing_skip", len(financing_df), "Financing classification skipped")
    building_csv = hpd_csv
else:
    output_path = Path(financing_output_path) if financing_output_path else Path(
        "data/processed/Affordable_Housing_Production_by_Building_with_financing.csv"
    )
    output_path.parent.mkdir(parents=True, exist_ok=True)

    print(f"Classifying financing types -> {output_path}")
    financing_df = query_and_add_financing(str(hpd_csv), output_path=str(output_path), use_cache=not refresh_ll44, use_eligibility_cache=not refresh_ll44_eligibility)
    building_csv = output_path

    # Record dataset after financing classification
    quality_tracker.analyze_hpd_data(financing_df, "Filtered_HPD")
    quality_tracker.record_pipeline_stage("after_financing", len(financing_df), "Added LL44 financing classification")

print(f"✅ Step 2 complete: {len(financing_df):,} records with financing classification")

# Display the dataframe with financing info
print("\n🔍 Financing Classification Results:")
print(f"Shape: {financing_df.shape}")

# Check if financing columns were added
financing_cols = [col for col in financing_df.columns if "financ" in col.lower() or "ll44" in col.lower()]
print(f"\nFinancing-related columns: {financing_cols}")

print("\n📊 Sample Data with Financing:")
display(financing_df.head())

# Show financing type distribution
if financing_cols:
    for col in financing_cols:
        if col in financing_df.columns:
            print(f"\n📈 Distribution of {col}:")
            display(financing_df[col].value_counts(dropna=False))


STEP 2: ADD FINANCING CLASSIFICATION
Classifying financing types -> data/processed/Affordable_Housing_Production_by_Building_with_financing.csv
Reading HPD data: data/raw/Affordable_Housing_Production_by_Building.csv
Loaded 8604 HPD buildings

Enriching HPD data with LL44 eligibility...
VERIFYING LL44 ELIGIBILITY CACHE
Local LL44 eligibility cache file not found at data/processed/ll44_eligibility_data.csv
Fetching fresh data from API...
Fetching LL44 eligibility data from NYC Open Data API...
Endpoint: https://data.cityofnewyork.us/resource/ucdy-byxd.json
Fetching records 1-1000...
  Retrieved 1000 records (total: 1,000)
Fetching records 1001-2000...
  Retrieved 14 records (total: 1,014)
Fetching records 1015-2014...

Completed! Retrieved 1,014 total records
Saved fresh data to: data/processed/ll44_eligibility_data.csv
Loaded 1014 LL44 eligible projects from eligibility dataset
VERIFYING LL44 FUNDING CACHE
Local LL44 cache file not found at data/processed/ll44_funding_data.csv
Fetchin

,Project ID,Project Name,Project Start Date,Building ID,Number,Street,Borough,Postcode,BBL,BIN,...,Counted Rental Units,Counted Homeownership Units,All Counted Units,Total Units,Program Group,Project Completion Date,Extended Affordability Status,Planned Tax Benefit,subject_to_ll44,Financing Type
0,44218,MEC E. 125TH ST. PARCEL B WEST,2018-12-31T00:00:00.000,987329,2319,3 AVENUE,Manhattan,10035,1017907501,1054682,...,297.0,NaN,297.0,404,Multifamily Finance Program,2022-02-16T00:00:00.000,No,Article XI,True,HPD Financed
1,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,927737,335,RALPH AVENUE,Brooklyn,11233,3015560003,3000000,...,NaN,13.0,13.0,13,Multifamily Finance Program,2024-09-24T00:00:00.000,No,Article XI,True,HPD Financed
2,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,969695,35,ROCHESTER AVENUE,Brooklyn,11233,3017090009,3000000,...,NaN,8.0,8.0,8,Multifamily Finance Program,2024-09-24T00:00:00.000,No,Article XI,True,HPD Financed
3,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,975702,18-22,SUYDAM PLACE,Brooklyn,11233,3017090028,3000000,...,NaN,15.0,15.0,15,Multifamily Finance Program,2024-09-24T00:00:00.000,No,Article XI,True,HPD Financed
4,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,977564,329,RALPH AVENUE,Brooklyn,11233,3015560007,3000000,...,NaN,10.0,10.0,10,Multifamily Finance Program,2024-09-24T00:00:00.000,No,Article XI,True,HPD Financed



📈 Distribution of subject_to_ll44:


subject_to_ll44
False    6114
True     2490
Name: count, dtype: int64


📈 Distribution of Financing Type:


Financing Type
Not Subject to LL44    6114
HPD Financed           2238
Privately Financed      252
Name: count, dtype: int64

In [12]:
# List all projects in the "Multifamily Finance Program" that are not HPD financed

# Filter multifamily finance projects that are NOT HPD financed
multifamily_not_hpd = financing_df[
    (financing_df['Program Group'] == 'Multifamily Finance Program') & 
    (financing_df['Financing Type'] != 'HPD Financed')
        &(financing_df['Reporting Construction Type'] == 'New Construction')

]

# Filter multifamily finance projects that ARE HPD financed
hpd_multifamily = financing_df[
    (financing_df['Program Group'] == 'Multifamily Finance Program') & 
    (financing_df['Financing Type'] == 'HPD Financed')
    &(financing_df['Reporting Construction Type'] == 'New Construction')
]

# Number of unique Project IDs for HPD financed multifamily projects
unique_proj_hpd_multifamily = hpd_multifamily['Project ID'].nunique()
print(f"Number of unique multifamily finance projects that are HPD financed: {unique_proj_hpd_multifamily}")

# Number of unique Project IDs for NOT HPD financed multifamily projects
unique_proj_multifamily_not_hpd = multifamily_not_hpd['Project ID'].nunique()
print(f"Number of unique multifamily finance projects that are NOT HPD financed: {unique_proj_multifamily_not_hpd}")

# Show dataframe for inspection
multifamily_not_hpd


Number of unique multifamily finance projects that are HPD financed: 329
Number of unique multifamily finance projects that are NOT HPD financed: 53


,Project ID,Project Name,Project Start Date,Building ID,Number,Street,Borough,Postcode,BBL,BIN,...,Counted Rental Units,Counted Homeownership Units,All Counted Units,Total Units,Program Group,Project Completion Date,Extended Affordability Status,Planned Tax Benefit,subject_to_ll44,Financing Type
75,44371,Navy Green (Building R2),2014-03-06T00:00:00.000,948016,8,VANDERBILT AVENUE,Brooklyn,11205,3020337502,3398205,...,NaN,75.0,75.0,99,Multifamily Finance Program,2016-03-25T00:00:00.000,No,421a,False,Not Subject to LL44
84,45647,HARLEN PARK 70 WEST 139TH STREET,2015-06-11T00:00:00.000,972438,70,WEST 139 STREET,Manhattan,10037,1017367502,1090308,...,NaN,42.0,42.0,64,Multifamily Finance Program,2018-10-10T00:00:00.000,No,421a,False,Not Subject to LL44
424,50410,CAMBA. 331 Saratoga Ave. Bergen Saratoga,2014-06-03T00:00:00.000,980814,331,SARATOGA AVENUE,Brooklyn,11233,3014470001,3413938,...,80.0,NaN,80.0,80,Multifamily Finance Program,2016-10-21T00:00:00.000,No,420c,True,Privately Financed
477,51123,BAM Cultural District North Site Tower II,2015-05-07T00:00:00.000,967799,15,LAFAYETTE AVENUE,Brooklyn,11217,3021077502,3397135,...,50.0,NaN,50.0,123,Multifamily Finance Program,2018-03-20T00:00:00.000,No,421a,True,Privately Financed
582,52082,ESSEX CROSSING - SITE 1,2015-11-24T00:00:00.000,968932,242,BROOME STREET,Manhattan,10002,1004097502,1089450,...,NaN,11.0,11.0,55,Multifamily Finance Program,2018-07-05T00:00:00.000,No,421a,False,Not Subject to LL44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8108,75925,RISEBORO. ALAFIA PH II,2024-12-20T00:00:00.000,1014053,879,ERSKINE STREET,Brooklyn,11239,NaN,NaN,...,425.0,NaN,425.0,425,Multifamily Finance Program,NaN,No,420c,False,Not Subject to LL44
8109,75925,RISEBORO. ALAFIA PH II,2024-12-20T00:00:00.000,1015567,888,FOUNTAIN AVENUE,Brooklyn,11239,3045860300,3327531,...,209.0,NaN,209.0,209,Multifamily Finance Program,NaN,No,420c,False,Not Subject to LL44
8335,76445,MOTHER GASTON-HINSDALE (FY25 SUBSIDY INCREASE),2025-06-05T00:00:00.000,1017498,1915,BERGEN STREET,Brooklyn,11233,3014460056,3000000,...,2.0,1.0,3.0,3,Multifamily Finance Program,NaN,No,NaN,False,Not Subject to LL44
8336,76445,MOTHER GASTON-HINSDALE (FY25 SUBSIDY INCREASE),2025-06-05T00:00:00.000,1017499,2178,DEAN STREET,Brooklyn,11233,3014470020,3000000,...,2.0,1.0,3.0,3,Multifamily Finance Program,NaN,No,NaN,False,Not Subject to LL44


In [7]:
# Convert example BBL 2025730071 to Borough, Block, Lot
example_bbl = "2025730071".zfill(10)  # Ensure 10 digits
# NYC BBL convention: [1 char boro][5 char block][4 char lot]
borough_num = example_bbl[0]
block = example_bbl[1:6].lstrip("0")
lot = example_bbl[6:].lstrip("0")
boro_map = {
    "1": "MANHATTAN",
    "2": "BRONX",
    "3": "BROOKLYN",
    "4": "QUEENS",
    "5": "STATEN ISLAND"
}
borough_name = boro_map.get(borough_num, "Unknown")
print(f"BBL: {example_bbl} -> Borough: {borough_name} (code {borough_num}), Block: {block}, Lot: {lot}")


BBL: 2025730071 -> Borough: BRONX (code 2), Block: 2573, Lot: 71


## 🏗️ Step 3A: Query DOB Filings

Search for DOB New Building filings.

**Depends on:** Step 2
**Options:**
- Set `skip_dob = True` to use existing DOB data
- Set `use_bbl_fallback = False` to disable BBL fallback

In [ ]:
# Step 3A Configuration
skip_dob = False  # Set to True to use existing DOB data
use_bbl_fallback = True  # Set to False to disable BBL fallback
dob_output_path = None  # Custom DOB output path

print("\n" + "=" * 70)
print("STEP 3A: QUERY DOB FILINGS")
print("=" * 70)

dob_output = Path(dob_output_path) if dob_output_path else Path(
    f"data/processed/{building_csv.stem}_dob_filings.csv"
)
dob_output.parent.mkdir(parents=True, exist_ok=True)

# Check for existing DOB files when skipping
if skip_dob:
    print("⏭️ Using existing DOB data")
    # Look for existing files
    alt_dob_path = Path(f"data/external/{building_csv.stem}_dob_filings.csv")
    if dob_output.exists():
        print(f"📁 Using existing DOB data at {dob_output}")
        dob_df = pd.read_csv(dob_output)
    elif alt_dob_path.exists():
        print(f"📁 Using existing DOB data from external folder: {alt_dob_path}")
        dob_output = alt_dob_path
        dob_df = pd.read_csv(dob_output)
    else:
        print("⚠️ No existing DOB data found")
        dob_df = None
        dob_output = None
else:
    print(f"🔍 Querying DOB APIs using {building_csv} -> {dob_output}")
    print("   This may take several minutes...")
    query_dob_filings(
        str(building_csv),
        output_path=str(dob_output),
        use_bbl_fallback=use_bbl_fallback,
    )
    print(f"✅ DOB query completed: {dob_output}")
    dob_df = pd.read_csv(dob_output)

# Display DOB data if available
if dob_df is not None:
    print(f"📊 DOB Filings Data: {dob_df.shape[0]} records")
    print("Columns:")
    for col in dob_df.columns:
        print(f"  - {col}")
    
    print("\n📊 Sample DOB Data:")
    display(dob_df.head())
    
    # Show some statistics
    if "filing_date" in dob_df.columns:
        print("\n📈 DOB Filing Date Statistics:")
        display(dob_df["filing_date"].describe())
else:
    print("⚠️ No DOB data available")

## 🏛️ Step 3B: Query Certificate of Occupancy

Search for Certificate of Occupancy filings.

**Depends on:** Step 2
**Options:**
- Set `skip_co = True` to use existing CO data

In [ ]:
# Step 3B Configuration
skip_co = False  # Set to True to use existing CO data
co_output_path = None  # Custom CO output path

print("\n" + "=" * 70)
print("STEP 3B: QUERY CERTIFICATE OF OCCUPANCY")
print("=" * 70)

# Generate BIN file for CO searches
bin_output = Path("data/processed/workflow_bins.txt")
bin_file = _write_bin_file(building_csv, bin_output)

print(f"\n📋 BIN file created: {bin_file}")
print(f"Contains {len(bin_file.read_text().split())} BINs")

co_output = Path(co_output_path) if co_output_path else Path(
    f"data/processed/{bin_file.stem}_co_filings.csv"
)
co_output.parent.mkdir(parents=True, exist_ok=True)

if skip_co:
    print("⏭️ Using existing CO data")
    # Look for existing CO files
    alt_co_path = Path(f"data/external/{bin_file.stem}_co_filings.csv")
    if co_output.exists():
        print(f"📁 Using existing CO data at {co_output}")
        co_df = pd.read_csv(co_output)
    elif alt_co_path.exists():
        print(f"📁 Using existing CO data from external folder: {alt_co_path}")
        co_output = alt_co_path
        co_df = pd.read_csv(co_output)
    else:
        print("⚠️ No existing CO data found")
        co_df = None
        co_output = None
else:
    print(f"🏛️ Querying CO APIs using {bin_file} -> {co_output}")
    query_co_filings(str(bin_file), output_path=str(co_output))
    co_df = pd.read_csv(co_output)

# Display CO data if available
if co_df is not None:
    print(f"📊 Certificate of Occupancy Data: {co_df.shape[0]} records")
    print("Columns:")
    for col in co_df.columns:
        print(f"  - {col}")
    
    print("\n📊 Sample CO Data:")
    display(co_df.head())
    
    # Show some statistics
    if "issue_date" in co_df.columns:
        print("\n📈 CO Issue Date Statistics:")
        display(co_df["issue_date"].describe())
else:
    print("⚠️ No CO data available")

## 📊 Step 4: Generate Timelines and Charts

Create timeline visualizations from enriched data.

**Depends on:** Steps 2, 3A
**Options:**
- Set `skip_join = True` to skip timeline creation
- Set `skip_charts = True` to skip chart generation

In [ ]:
# Step 4 Configuration
skip_join = False   # Set to True to skip timeline creation
skip_charts = False # Set to True to skip chart generation

print("\n" + "=" * 70)
print("STEP 4: GENERATE TIMELINES AND CHARTS")
print("=" * 70)

if skip_join:
    print("⏭️ Skipping timeline join step.")
else:
    if dob_output is None or not dob_output.exists():
        print("⚠️ No DOB data available; skipping timeline creation.")
    else:
        print("🔗 Building timelines...")
        create_separate_timelines(
            str(building_csv),
            str(dob_output),
            str(co_output) if co_output else None,
        )
        
        # Load and display timeline data
        hpd_timeline = Path(str(building_csv).replace(".csv", "_hpd_financed_timeline.csv"))
        private_timeline = Path(str(building_csv).replace(".csv", "_privately_financed_timeline.csv"))
        
        if hpd_timeline.exists():
            hpd_timeline_df = pd.read_csv(hpd_timeline)
            print(f"\n📊 HPD Financed Timeline Data ({hpd_timeline_df.shape[0]} records):")
            display(hpd_timeline_df.head())
            
            # Show event type distribution
            if "event_type" in hpd_timeline_df.columns:
                print("\n📈 Event Types in HPD Timeline:")
                display(hpd_timeline_df["event_type"].value_counts())
        
        if private_timeline.exists():
            private_timeline_df = pd.read_csv(private_timeline)
            print(f"\n📊 Privately Financed Timeline Data ({private_timeline_df.shape[0]} records):")
            display(private_timeline_df.head())
            
            # Show event type distribution
            if "event_type" in private_timeline_df.columns:
                print("\n📈 Event Types in Private Timeline:")
                display(private_timeline_df["event_type"].value_counts())

if skip_charts:
    print("⏭️ Skipping chart generation.")
else:
    # Charts
    print("\n📈 Generating charts...")
    default_timeline_stem = "Affordable_Housing_Production_by_Building_with_financing"
    if Path(building_csv).name == f"{default_timeline_stem}.csv":
        create_financing_charts()
        print("✅ Created financing-specific charts")
    else:
        hpd_timeline = Path(str(building_csv).replace(".csv", "_hpd_financed_timeline.csv"))
        private_timeline = Path(str(building_csv).replace(".csv", "_privately_financed_timeline.csv"))
        
        if hpd_timeline.exists():
            create_timeline_chart(str(hpd_timeline))
            print(f"✅ Created HPD financed timeline chart")
        else:
            print(f"⚠️ No HPD financed timeline found; skipping.")

        if private_timeline.exists():
            create_timeline_chart(str(private_timeline))
            print(f"✅ Created privately financed timeline chart")
        else:
            print(f"⚠️ No privately financed timeline found; skipping.")

print("\n✅ Step 4 complete")

## 📋 Final Summary

Generate data quality report and workflow summary.

**Optional:** Run this at the end to see final statistics.

In [ ]:
print("\n" + "=" * 70)
print("📊 FINAL DATA QUALITY REPORT")
print("=" * 70)

# Generate final data quality report and Sankey diagram
quality_tracker.end_processing()
report_filename = quality_tracker.save_report_to_file("notebook_workflow")
sankey_filename = quality_tracker.generate_sankey_diagram()
quality_tracker.print_report()

print("\n🎉 WORKFLOW COMPLETED!")
print(f"📊 Data quality report: {report_filename}")
if sankey_filename:
    print(f"📊 Sankey diagram: {sankey_filename}")

# Summary of what we accomplished
print("\n📋 WORKFLOW SUMMARY:")
try:
    print(f"• HPD Records Processed: {len(hpd_df):,}")
except NameError:
    print("• HPD Records: Step 1 not run")
try:
    print(f"• Records with Financing: {len(financing_df):,}")
except NameError:
    print("• Records with Financing: Step 2 not run")
try:
    if dob_df is not None:
        print(f"• DOB Filings Found: {len(dob_df):,}")
    else:
        print("• DOB Filings: No data")
except NameError:
    print("• DOB Filings: Step 3A not run")
try:
    if co_df is not None:
        print(f"• CO Filings Found: {len(co_df):,}")
    else:
        print("• CO Filings: No data")
except NameError:
    print("• CO Filings: Step 3B not run")

print("\n✅ Notebook workflow complete!")
print("Each step showed dataframe views for inspection.")